In [1]:
#%matplotlib widget
import backtrader as bt
import backtrader.analyzers as btanalyzers
import rba_tools as rba
import pandas as pd
#import matplotlib
from datetime import datetime

In [2]:
class MaCrossStrategy(bt.Strategy):

    def __init__(self):
        ma_fast = bt.ind.SMA(period = 10)
        ma_slow = bt.ind.SMA(period = 50)

        self.crossover = bt.ind.CrossOver(ma_fast, ma_slow)

    def next(self):
        if not self.position:
            if self.crossover > 0:
                self.buy()
        elif self.crossover < 0:
            self.close()

In [3]:
cerebro = bt.Cerebro()

In [4]:
binance = rba.getBinanceExchange()
dataframe = rba.get_DataFrame(['ETH/BTC'], binance, '1/1/18', '12/31/19')
data = bt.feeds.PandasData(dataname=dataframe,
                               # datetime='Date',
                               nocase=True,
                               )
cerebro.adddata(data)

In [5]:
dataframe2 = rba.get_DataFrame(['LTC/BTC'], binance, '1/1/18', '12/31/19')
data2 = bt.feeds.PandasData(dataname=dataframe2,
                               # datetime='Date',
                               nocase=True,
                               )
cerebro.adddata(data2)

In [6]:
cerebro.addstrategy(MaCrossStrategy)

0

In [7]:
cerebro.broker.setcash(1000000.0)

In [8]:
cerebro.addsizer(bt.sizers.PercentSizer, percents = 10)

In [9]:
cerebro.addanalyzer(btanalyzers.SharpeRatio, _name = 'sharpe')
cerebro.addanalyzer(btanalyzers.Transactions, _name = 'trans')
cerebro.addanalyzer(btanalyzers.TradeAnalyzer, _name = 'trades')

In [10]:
back = cerebro.run()

In [11]:
cerebro.broker.getvalue()

971104.7165789388

In [12]:
back[0].analyzers.sharpe.get_analysis()

OrderedDict([('sharperatio', -0.8928682155011799)])

In [13]:
back[0].analyzers.trans.get_analysis()

OrderedDict([(datetime.datetime(2018, 4, 22, 0, 0),
              [[1472580.5501560937, 0.067836, 0, '', -99893.97420038877]]),
             (datetime.datetime(2018, 6, 6, 0, 0),
              [[-1472580.5501560937, 0.079898, 0, '', 117656.24079637157]]),
             (datetime.datetime(2018, 6, 8, 0, 0),
              [[1293250.4848865066, 0.078696, 0, '', -101773.64015862852]]),
             (datetime.datetime(2018, 6, 13, 0, 0),
              [[-1293250.4848865066, 0.075564, 0, '', 97723.17963996399]]),
             (datetime.datetime(2018, 11, 13, 0, 0),
              [[3072785.104811514, 0.032987, 0, '', -101361.96225241742]]),
             (datetime.datetime(2018, 11, 18, 0, 0),
              [[-3072785.104811514, 0.03131, 0, '', 96208.90163164849]]),
             (datetime.datetime(2018, 12, 26, 0, 0),
              [[2960254.6094997046, 0.034069, 0, '', -100852.91429104544]]),
             (datetime.datetime(2019, 1, 29, 0, 0),
              [[-2960254.6094997046, 0.030718, 0, 

In [14]:
back[0].analyzers.trades.get_analysis()

AutoOrderedDict([('total',
                  AutoOrderedDict([('total', 8), ('open', 0), ('closed', 8)])),
                 ('streak',
                  AutoOrderedDict([('won',
                                    AutoOrderedDict([('current', 0),
                                                     ('longest', 1)])),
                                   ('lost',
                                    AutoOrderedDict([('current', 7),
                                                     ('longest', 7)]))])),
                 ('pnl',
                  AutoOrderedDict([('gross',
                                    AutoOrderedDict([('total',
                                                      -28895.28342106143),
                                                     ('average',
                                                      -3611.910427632679)])),
                                   ('net',
                                    AutoOrderedDict([('total',
                                    

In [15]:
myplot = cerebro.plot(iplot=False)

In [20]:
back[0].data.close[0]

0.017953